In [1]:
!pip install pypdf nbformat langchain langchain-community langchain-openai langchain-google-genai transformers pyodbc


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install deep-translator



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install deep-translator pypdf python-pptx langchain transformers faiss-cpu sentence-transformers



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# %%
from deep_translator import GoogleTranslator
from pypdf import PdfReader
from pptx import Presentation
import json
from nbformat import read
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import BartForConditionalGeneration, BartTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate
import os
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [5]:
from flask import Flask, render_template, request
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from core import return_content, split_text, model_open_Ai  
import sqlite3
from datetime import datetime
import os

In [6]:
DB_PATH = "chat_history_sqlserver"

conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS history (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    question TEXT,
    answer TEXT,
    created_at TEXT
)
""")
conn.commit()

def save_history(q, a):
    cur.execute(
        "INSERT INTO history (question, answer, created_at) VALUES (?,?,?)",
        (q, a, datetime.now().isoformat())
    )
    conn.commit()

def get_history(limit: int = 20):
    cur.execute("SELECT question, answer FROM history ORDER BY id DESC LIMIT ?", (limit,))
    return cur.fetchall()


In [7]:
FOLDER_PATH = "./data"

all_text = ""

for file in os.listdir(FOLDER_PATH):
    full_path = os.path.join(FOLDER_PATH, file)
    if os.path.isfile(full_path):
        all_text += return_content(full_path) + "\n\n"

docs = split_text(all_text)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embeddings)
qa_chain = model_open_Ai(embeddings, db)

C:\Users\allal\AppData\Local\Temp\ipykernel_25348\1701902190.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
app = Flask(__name__)

@app.route("/", methods=["GET","POST"])
@app.route("/", methods=["GET","POST"])
def home():
    answer = None
    if request.method == "POST":
        user_query = request.form.get("query")
        try:
            resp = qa_chain.invoke({"query": user_query})
            answer = resp["result"]
            save_history(user_query, answer)
        except:
            answer = "Invalid API key or connection error"

    history = get_history()
    return render_template("index.html", answer=answer, history=history)


if __name__ == "__main__":
    app.run(debug=True, port=5000 , use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Nov/2025 22:25:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2025 22:25:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/Nov/2025 22:26:11] "POST / HTTP/1.1" 200 -
